In [1]:
import pymoo as pm
import numpy as np
import pandas as pd


from pymoo.operators.sampling.rnd import FloatRandomSampling

from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.termination import get_termination

from astropy import units as u
from astropy.time import Time

from poliastro.twobody import Orbit
from poliastro.bodies import Sun, Earth

from poliastro.ephem import Ephem

from poliastro.iod import izzo

import datetime
from datetime import timedelta

In [2]:
# Set some constants
consts = {
    'G': 6.67430e-20 *((u.km**3)/(u.kg * u.s**2)), # km^3/(kg * s^2)
    'M_earth': 5.9722e24 *(u.kg), # kg
    'R_earth': 6371 *(u.km), # m
    'M_Sun': 1.9891e30 *(u.kg) # kg
}

In [3]:
# Functions for determining orbital elements
def eccentricity(init_pos,init_vel,gravitational_parameter):
    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)
    unit_vec_pos = init_pos/np.linalg.norm(init_pos)
    e_vec = (1/gravitational_parameter)*(np.cross(init_vel,sp_angular_momentum)) - unit_vec_pos
    return np.linalg.norm(e_vec).value

def semi_major_axis(init_pos,init_vel,gravitational_parameter):

    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)
    h = np.linalg.norm(sp_angular_momentum)

    unit_vec_pos = init_pos/np.linalg.norm(init_pos)
    e_vec = (1/gravitational_parameter)*(np.cross(init_vel,sp_angular_momentum)) - unit_vec_pos
    e = np.linalg.norm(e_vec).value

    s_major_axis = ((h**2)/gravitational_parameter) * (1/(1-e**2))

    return s_major_axis

def inclination(init_pos, init_vel):
    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)
    h = np.linalg.norm(sp_angular_momentum)
    h_z = sp_angular_momentum[2].copy()

    inc = np.arccos(h_z/h).to(u.degree)
    return inc

def long_of_ascending_node(init_pos,init_vel):
    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)

    nodeline = np.cross([0,0,1],sp_angular_momentum).to(u.km**2/u.s)
    nodeline_abs = np.linalg.norm(nodeline)
    nodeline_x = nodeline[0].copy()
    nodeline_y = nodeline[1].copy()

    if nodeline_y >= 0:
        ascending_node =np.arccos(nodeline_x/ nodeline_abs).to(u.degree)
    else:
        ascending_node = 360*u.degree - np.arccos(nodeline_x/ nodeline_abs).to(u.degree)
        
    return ascending_node


def argument_of_periapsis(init_pos,init_vel,gravitational_parameter):
    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)

    nodeline = np.cross([0,0,1],sp_angular_momentum).to(u.km**2/u.s)
    nodeline_abs = np.linalg.norm(nodeline)

    unit_vec_pos = init_pos/np.linalg.norm(init_pos)
    e_vec = (1/gravitational_parameter)*(np.cross(init_vel,sp_angular_momentum)) - unit_vec_pos
    e = np.linalg.norm(e_vec).value

    if e_vec[2] >= 0:
        arg_peri = np.arccos(np.dot(nodeline,e_vec)/(nodeline_abs*e))
        arg_peri = arg_peri.to(u.degree)
    else:
        temp = np.arccos(np.dot(nodeline,e_vec)/(nodeline_abs*e))
        temp = temp.to(u.degree)
        arg_peri = 360*u.degree - temp

    return arg_peri

def true_anomaly(init_pos, init_vel, gravitational_parameter):
    sp_angular_momentum = np.cross(init_pos,init_vel).to(u.km**2/u.s)

    unit_vec_pos = init_pos/np.linalg.norm(init_pos)
    e_vec = (1/gravitational_parameter)*(np.cross(init_vel,sp_angular_momentum)) - unit_vec_pos
    e = np.linalg.norm(e_vec).value

    temp = np.dot(e_vec,init_pos)
    dist = np.linalg.norm(init_pos)

    if np.dot(init_pos,init_vel) >= 0:
        theta = np.arccos(temp/(e*dist)).to(u.degree)
    else:
        tmp_2 = np.arccos(temp/(e*dist)).to(u.degree)
        theta = 360*u.degree -tmp_2

    return theta

In [4]:
# Functions 
# Pull data for Earth give CubeSat details and Apophis details (like pos, vel and orbit object)
# a here is altitude of initial orbit not semi-major axis
def Earth_Apophis_details(start_date,end_date,  a,inc,raan,argp,nu):
    # Uses HCRS 
        # Heliocentric system with axis aligned to the ICRS 
        # Origin is at the Suns centre of mass rather than solar system barycenter 

    # Set epoch for Earth (start of mission)
    start_year=str(start_date)[0:4]
    start_month=str(start_date)[5:7]
    start_day=str(start_date)[8:10]
    start_time='12:00:00'
    t_start = Time(f'{start_year}-{start_month}-{start_day}T{start_time}',format='isot', scale='utc')

    # set epoch for Apophis (arrival)
    end_year=str(end_date)[0:4]
    end_month=str(end_date)[5:7]
    end_day=str(end_date)[8:10]
    end_time='12:00:00'
    t_end = Time(f'{end_year}-{end_month}-{end_day}T{end_time}',format='isot', scale='utc')

    # Import Ephem from JPL_Horizons for Apophis and Earth
    apophis_ephem_end = Ephem.from_horizons('Apophis',t_end)  
    earth_ephem_start = Ephem.from_horizons('399', t_start)

    # Set orbit with Sun as attractor for both bodies at chosen epoch (Orbit propagator)
    Apophis_orbit = Orbit.from_ephem(Sun, apophis_ephem_end,t_end)
    Earth_orbit = Orbit.from_ephem(Sun, earth_ephem_start, t_start)

    # Define an orbit for the sattelite
    ecc=0 *u.one
    a = a *u.km
    inc = inc *u.degree
    raan = raan *u.degree
    argp = argp *u.degree
    nu = nu *u.degree

    CubeSat_in_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, t_start)

    # Change units
    pos_earth = Earth_orbit.r.to(u.km)
    vel_earth = Earth_orbit.v.to(u.km / u.s)
    pos_apophis = Apophis_orbit.r.to(u.km)
    vel_apophis = Apophis_orbit.v.to(u.km / u.s)

    pos_CubeSat = CubeSat_in_orbit.r.to(u.km) + pos_earth
    vel_CubeSat = CubeSat_in_orbit.v.to(u.km / u.s) + vel_earth

    return [pos_CubeSat,vel_CubeSat,Earth_orbit, pos_apophis, vel_apophis, Apophis_orbit]

# Time of transfer trajectory
def time_of_flight(start_date,end_date):

    tof=int((end_date-start_date).total_seconds())
    return tof*u.s


# Find Number of Days for mission
def time_of_flight_days(start_date, end_date):
    tmp = ''
    i = 0
    while str(end_date-start_date)[i] != ' ':
        tmp = tmp + str(end_date-start_date)[i]
        i += 1
    return int(tmp)

# Use position and velocity + Lambert solver to find L2 norm of v_inf(s)
def v_inf_calc(start_date,end_date, a,inc,raan,argp,nu):
    info = Earth_Apophis_details(start_date,end_date, a,inc,raan,argp,nu)
    pos_CubeSat_1,vel_CubeSat_1,Earth_orbit, pos_apophis,vel_apophis,Apophis_orbit = info[0], info[1], info[2], info[3], info[4], info[5]
        # ERFA warning due to limitation of the conversion to datetime (cannot compute leapseconds etc)

    total_time=time_of_flight(start_date,end_date)
    if total_time > 0:
        sol = izzo.lambert(k = consts['G']*consts['M_Sun'], r0 = pos_CubeSat_1.to(u.km), r = pos_apophis.to(u.km), tof = total_time, prograde=True)
        V_inf_earth = sol[0] - vel_CubeSat_1
        V_inf_Apophis = sol[1] - vel_apophis
        V_inf_earth_norm = np.linalg.norm(V_inf_earth)
        V_inf_Apophis_norm = np.linalg.norm(V_inf_Apophis) 


        # Return norms of vinfs, and vectors of vinfs
        return [V_inf_earth_norm.to(u.km / u.s), V_inf_Apophis_norm.to(u.km / u.s),
                V_inf_earth, V_inf_Apophis,
                sol[0],sol[1],]
    else:
        # "penalty function so lamberst solver doesnt revieve an error"
        return[9999999999999999, 9999999999999999]
    
# Removes units from a value and return 
def remove_units(value):
    number = str(value)
    ref_sym = ['0','1','2','3','4','5','6','7','8','9','.']
    count = 0
    for i in range(len(number)):
        count+=1
        if number[i] in ref_sym:
            continue
        else:
            break
    return float(number[0:count])

# Convert from float to date given a reference date which is the start window
# Used to change dates in a column
def Convert_to_date(ref_date,data_frame,column_name):
    temp = np.array(data_frame[column_name])
    col = []
    for i in range(len(temp)):
        col.append(int(temp[i]))
    for i in range(len(col)):
        col[i] = str(ref_date + timedelta(int(col[i])))
    data_frame[column_name] = col



In [5]:
# # Vepa suggestion

# def Earth_Apophis_details(start_date,end_date,  a,inc,raan,argp,nu):
#     # Uses HCRS 
#         # Heliocentric system with axis aligned to the ICRS 
#         # Origin is at the Suns centre of mass rather than solar system barycenter 

#     # Set epoch for Earth (start of mission)
#     start_year=str(start_date)[0:4]
#     start_month=str(start_date)[5:7]
#     start_day=str(start_date)[8:10]
#     start_time='12:00:00'
#     t_start = Time(f'{start_year}-{start_month}-{start_day}T{start_time}',format='isot', scale='utc')

#     # set epoch for Apophis (arrival)
#     end_year=str(end_date)[0:4]
#     end_month=str(end_date)[5:7]
#     end_day=str(end_date)[8:10]
#     end_time='12:00:00'
#     t_end = Time(f'{end_year}-{end_month}-{end_day}T{end_time}',format='isot', scale='utc')

#     # Import Ephem from JPL_Horizons for Apophis and Earth
#     apophis_ephem_end = Ephem.from_horizons('Apophis',t_end)  
#     earth_ephem_start = Ephem.from_horizons('399', t_start)

#     # Set orbit with Sun as attractor for both bodies at chosen epoch (Orbit propagator)
#     Apophis_orbit = Orbit.from_ephem(Sun, apophis_ephem_end,t_end)
#     Earth_orbit = Orbit.from_ephem(Sun, earth_ephem_start, t_start)

#     # Define an orbit for the sattelite
#     ecc=0 *u.one
#     a = a *u.km
#     inc = inc *u.degree
#     raan = raan *u.degree
#     argp = argp *u.degree
#     nu = nu *u.degree

#     CubeSat_in_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, t_start)

#     # Change units
#     pos_earth = Earth_orbit.r.to(u.km)
#     vel_earth = Earth_orbit.v.to(u.km / u.s)
#     pos_apophis = Apophis_orbit.r.to(u.km)
#     vel_apophis = Apophis_orbit.v.to(u.km / u.s)

#     pos_CubeSat = CubeSat_in_orbit.r.to(u.km) + pos_earth
#     vel_CubeSat = vel_earth


#     return [pos_CubeSat,vel_CubeSat,Earth_orbit, pos_apophis, vel_apophis, Apophis_orbit]

In [6]:
# Set mission start date, end date and mission length in days
start_date=datetime.date(2027,1,1)
end_date=datetime.date(2029,1,1)

time_of_flight_days(start_date,end_date)

731

In [7]:
# Problem formulation
class BallisticTrajectory(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=7,
                        n_obj=1, 
                        n_ieq_constr=1, 
                        xl=np.array([0, 0,  160,  0, 0, 0, 0]), 
                        xu=np.array([731, 731,  1000,  180, 360, 360, 360])
                        )
        self.ref_date = datetime.date(2027,1,1)

        self.consts_2 = {
            'G': 6.67430e-20,       # *((u.km**3)/(u.kg * u.s**2)), # km^3/(kg * s^2)
            'M_earth': 5.9722e24,   # *(u.kg),                      # kg
            'R_earth': 6371,        # *(u.km),                      # km
            'M_Sun': 1.9891e30,     # *(u.kg),                      # kg
            }
    
    def _evaluate(self, x, out, *arg, **kwargs):
    
        # Convertback to datetime objects
        start_date = self.ref_date + timedelta(x[0])
        end_date = self.ref_date + timedelta(x[1])

        # Worked out values
        V_inf_earth_norm=remove_units(v_inf_calc(start_date, end_date, x[2]+self.consts_2['R_earth'],x[3],x[4],x[5],x[6])[0])
        V_inf_Apophis_norm=remove_units(v_inf_calc(start_date, end_date, x[2]+self.consts_2['R_earth'],x[3],x[4],x[5],x[6])[1])
        gravitational_param_earth = self.consts_2['G']*self.consts_2['M_earth']
        delta_v_1 = np.sqrt(V_inf_earth_norm**2 + (2*gravitational_param_earth)/(self.consts_2['R_earth']+x[2]))-np.sqrt(gravitational_param_earth/(self.consts_2['R_earth']+x[2]))
        delta_v_2 = V_inf_Apophis_norm

        # Objective function:
        f1 = delta_v_1 + delta_v_2
        g1 = x[0] - x[1]


        out["F"] = np.array([f1])
        out["G"] = np.array([g1])


In [8]:
# PSO parameters

problem = BallisticTrajectory()


algorithm = PSO(
    pop_size=100,
    sampling=FloatRandomSampling(),
    eliminate_duplicates=True
)
termination = get_termination("n_gen", 150)

res = minimize(problem,
               algorithm,
               termination,
               seed=3,
               verbose=True)

/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/astropy/units/decorators.py:302: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


n_gen  |  n_eval  |    f     |    S    |    w    |    c1    |    c2    |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |        - |       - |  0.9000 |  2.00000 |  2.00000 |  0.000000E+00 |  1.112922E+02 |  5.406301E+01 |  6.1807550680
     2 |      200 |  0.20394 |       3 |  0.5312 |  2.00000 |  2.01916 |  0.000000E+00 |  5.303698E+01 |  5.109086E+01 |  6.0786489166
     3 |      300 |  0.09498 |       3 |  0.4605 |  1.99065 |  2.03371 |  0.000000E+00 |  8.6571321021 |  5.291985E+01 |  5.0314155201
     4 |      400 |  0.05194 |       3 |  0.4328 |  1.97892 |  2.04224 |  0.000000E+00 |  0.2861238067 |  9.7372519290 |  5.0314155201
     5 |      500 |  0.02264 |       3 |  0.4142 |  1.96897 |  2.05118 |  0.000000E+00 |  0.000000E+00 |  8.6575324909 |  5.0314155201
     6 |      600 |  0.01085 |       3 |  0.4068 |  1.95977 |  2.06187 |  0.000000E+00 |  0.000000E+00 |  7.8233265498 |  5.0314155201
     7 |      700 | -1.4E-03 |       3 |  0.3991 |  1.9

In [12]:
# Results from the Optimisation
results = pd.DataFrame(res.pop.get('X'), columns=['Start','End','Altitude [km]', 'inc', 'argp', 'raan', 'nu'])
results['Delta-V [km/s]'] = res.pop.get('F')
results.head()

,Start,End,Altitude [km],inc,argp,raan,nu,Delta-V [km/s]
0,344.757596,615.735760,999.979940,66.850749,253.279931,210.418717,225.388130,4.931931
1,344.405000,615.409653,999.999336,63.628197,254.312145,204.128139,231.379783,4.931876
2,344.724307,615.492260,999.988031,59.623083,255.065771,203.647174,231.609803,4.931863
3,344.906936,615.858707,999.993460,66.176871,253.352452,204.676064,231.033435,4.931951
4,344.381865,615.158025,999.438502,75.661992,250.636568,206.165421,230.595489,4.932197


In [13]:
def Return_df(ref_date):

    data_frame = pd.DataFrame(res.pop.get('X'), columns=['Start [YYYY-MM-DD]','End [YYYY-MM-DD]',
                                                         'Parking Orbit Alt [km]',
                                                         'inc [deg]',
                                                         'raan [deg]',
                                                         'argp [deg]',
                                                         'nu [deg]'])
    data_frame['Delta-V [km/s]'] = res.pop.get('F')

    # grv_par_sun = 1.32712440018*10**11 *u.km**3/u.s**2
    grv_par_sun = consts['G']*consts['M_Sun']
    grv_par_earth = consts['G']*consts['M_earth']

    # Convert Start dates and add V1
    temp = np.array(data_frame['Start [YYYY-MM-DD]'])
    col = []
    start_dates = []
    end_dates = []
    for i in range(len(temp)):
        col.append(int(temp[i]))
    for i in range(len(col)):
        start_dates.append(ref_date + timedelta(int(col[i])))
        col[i] = str(ref_date + timedelta(int(col[i])))
    data_frame['Start [YYYY-MM-DD]'] = col
    
    # Convert End dates and add V2
    temp = np.array(data_frame['End [YYYY-MM-DD]'])
    col = []
    for i in range(len(temp)):
        col.append(int(temp[i]))
    for i in range(len(col)):
        end_dates.append(ref_date + timedelta(int(col[i])))
        col[i] = str(ref_date + timedelta(int(col[i])))
    data_frame['End [YYYY-MM-DD]'] = col

    # caluculate the v_infs
    v1 = []
    v2 = []

    for i in range(len(start_dates)):
        temp_2 = v_inf_calc(start_dates[i],end_dates[i],(data_frame['Parking Orbit Alt [km]'][i] + remove_units(consts['R_earth'])),
                          data_frame['inc [deg]'][i], data_frame['raan [deg]'][i], data_frame['argp [deg]'][i],data_frame['nu [deg]'][i])
        V_inf_earth_norm=temp_2[0]
        V_inf_Apophis_norm=temp_2[1]

        delta_v_1 = np.sqrt(V_inf_earth_norm**2 + (2*grv_par_earth)/(consts['R_earth']+(data_frame['Parking Orbit Alt [km]'][i]*u.km)))-np.sqrt(grv_par_earth/(consts['R_earth']+(data_frame['Parking Orbit Alt [km]'][i]*u.km)))
        delta_v_2 = V_inf_Apophis_norm
        

        v1.append(delta_v_1.value)
        v2.append(delta_v_2.value)


    data_frame['Delta-V1 [km/s]'] = v1
    data_frame['Delta-V2 [km/s]'] = v2


    data_frame[['ecc_tr','a_tr [km]','inc_tr [deg]','raan_tr [deg]','argp_tr [deg]','nu_tr [deg]']] = 0
    for i in range(len(start_dates)):
        init_pos, init_vel, trsh, trsh, trsh, trsh = Earth_Apophis_details(data_frame['Start [YYYY-MM-DD]'][i], 
                                                                           data_frame['End [YYYY-MM-DD]'][i], 
                                                                           (data_frame['Parking Orbit Alt [km]'][i]*u.km + consts['R_earth']).to(u.km).value, 
                                                                           data_frame['inc [deg]'][i],
                                                                           data_frame['raan [deg]'][i],
                                                                           data_frame['argp [deg]'][i],
                                                                           data_frame['nu [deg]'][i]
                                                                           )
        temp_3 = v_inf_calc(start_dates[i],end_dates[i],(data_frame['Parking Orbit Alt [km]'][i] + remove_units(consts['R_earth'])),
                          data_frame['inc [deg]'][i], data_frame['raan [deg]'][i], data_frame['argp [deg]'][i],data_frame['nu [deg]'][i])
        
        init_vel = temp_3[4].to(u.km/u.s)


        data_frame['ecc_tr'][i] = eccentricity(init_pos,init_vel,grv_par_sun)
        data_frame['a_tr [km]'][i] = semi_major_axis(init_pos,init_vel,grv_par_sun).to(u.km).value
        data_frame['inc_tr [deg]'][i] = inclination(init_pos,init_vel).value
        data_frame['raan_tr [deg]'][i] = long_of_ascending_node(init_pos,init_vel).value
        data_frame['nu_tr [deg]'][i] = true_anomaly(init_pos,init_vel,grv_par_sun).value
        data_frame['argp_tr [deg]'][i] = argument_of_periapsis(init_pos,init_vel,grv_par_sun).value

    # Ensure no duplicates
    data_frame = data_frame.drop_duplicates().reset_index(drop=True)

    return data_frame

ref_date = start_date
df = Return_df(ref_date) 
df.head()

/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/astropy/units/decorators.py:302: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)
/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/emilsieciechowicz/anaconda3/lib/python3.11/site-packages/astropy/units/decorators.py:302: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)
/var/folders/vh/3wz49ynx7fdd5w428wl350zh0000gn/T/ipykernel_83588/866742632.py:75: SettingWithCopyWarning: 
A value is trying to be set on 

,Start [YYYY-MM-DD],End [YYYY-MM-DD],Parking Orbit Alt [km],inc [deg],raan [deg],argp [deg],nu [deg],Delta-V [km/s],Delta-V1 [km/s],Delta-V2 [km/s],ecc_tr,a_tr [km],inc_tr [deg],raan_tr [deg],argp_tr [deg],nu_tr [deg]
0,2027-12-11,2028-09-07,999.979940,66.850749,253.279931,210.418717,225.388130,4.931931,3.054624,1.877307,0.219817,1.432163e+08,23.115293,354.697672,334.552809,109.051740
1,2027-12-11,2028-09-07,999.999336,63.628197,254.312145,204.128139,231.379783,4.931876,3.054603,1.877273,0.219817,1.432163e+08,23.115239,354.697698,334.552702,109.051667
2,2027-12-11,2028-09-07,999.988031,59.623083,255.065771,203.647174,231.609803,4.931863,3.054638,1.877225,0.219817,1.432162e+08,23.115163,354.697734,334.552590,109.051562
3,2027-12-11,2028-09-07,999.993460,66.176871,253.352452,204.676064,231.033435,4.931951,3.054650,1.877301,0.219817,1.432163e+08,23.115283,354.697677,334.552807,109.051707
4,2027-12-11,2028-09-07,999.438502,75.661992,250.636568,206.165421,230.595489,4.932197,3.054830,1.877367,0.219818,1.432164e+08,23.115388,354.697627,334.553018,109.052009


In [11]:
# # Load the existing Excel file
# existing_excel_file = '/Users/emilsieciechowicz/Documents/QMUL/Fourth Year/Project/CubeSat/Calculations/Optimisation Algos/Results.xlsx'

# # Open the existing Excel file in append mode and write the new DataFrame as a new sheet
# with pd.ExcelWriter(existing_excel_file, mode='a', engine='openpyxl') as writer:
#     df.to_excel(writer, sheet_name='M2- final', index=True)